## Постановка задачи
Загрузим данные и разделим выборку на обучающую/проверочную в соотношении 80/20.

Применим метод ближайших соседей (kNN) для классификации скоринга. Будем использовать только биометрические данные.

Проверим качество предсказания через каппа-метрику и матрицу неточностей.

Данные:
* https://video.ittensive.com/machine-learning/prudential/train.csv.gz

Соревнование: https://www.kaggle.com/c/prudential-life-insurance-assessment/

### Подключение библиотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

### Загрузка данных

In [2]:
data = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/train.csv.gz")
print (data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 128 entries, Id to Response
dtypes: float64(18), int64(109), object(1)
memory usage: 58.0+ MB
None


### Разделение данных

In [3]:
data_train, data_test = train_test_split(data, test_size=0.2)
print (data_train.head())

          Id  Product_Info_1 Product_Info_2  Product_Info_3  Product_Info_4  \
1871    2505               1             D4              26        0.230769   
17271  23020               1             A1              26        0.076923   
2981    3972               1             D4              10        0.230769   
50679  67485               1             D3              26        1.000000   
4969    6612               1             D3              26        0.698247   

       Product_Info_5  Product_Info_6  Product_Info_7   Ins_Age        Ht  \
1871                2               3               1  0.432836  0.600000   
17271               2               3               1  0.552239  0.672727   
2981                2               3               1  0.283582  0.636364   
50679               2               3               1  0.417910  0.781818   
4969                2               3               1  0.388060  0.672727   

       ...  Medical_Keyword_40  Medical_Keyword_41  Medical_Ke

### Расчет модели kNN (k ближайших соседей)
Вычисляем не центры (кластеры) исходных групп, а расстояние до всех значений. Выбираем то значение, которое превалирует у k ближайших соседей.

Для оценки качества модели возьмем k равным 10, 100, 1000, 10000.

In [4]:
columns = ["Wt", "Ht", "Ins_Age", "BMI"]
max_nn = data_train.groupby("Response").count()["Id"].min()
knn10 = KNeighborsClassifier(n_neighbors=10)
knn100 = KNeighborsClassifier(n_neighbors=100)
knn1000 = KNeighborsClassifier(n_neighbors=1000)
knn10000 = KNeighborsClassifier(n_neighbors=10000)
knnmax = KNeighborsClassifier(n_neighbors=max_nn)

In [6]:
y = data_train["Response"]
x = pd.DataFrame(data_train, columns=columns)
knn10.fit(x, y)
knn100.fit(x, y)
knn1000.fit(x, y)
knn10000.fit(x, y)
knnmax.fit(x, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=801, p=2,
                     weights='uniform')

### Предсказание данных
Внимание: 10000 соседей потребует порядка 4 Гб оперативной памяти

In [7]:
data_test = pd.DataFrame(data_test)
x_test = pd.DataFrame(data_test, columns=columns)
data_test["target_10"] = knn10.predict(x_test)
data_test["target_100"] = knn100.predict(x_test)
data_test["target_1000"] = knn1000.predict(x_test)
data_test["target_10000"] = knn10000.predict(x_test)
data_test["target_max"] = knnmax.predict(x_test)
print (data_test.head(20))

          Id  Product_Info_1 Product_Info_2  Product_Info_3  Product_Info_4  \
7633   10172               1             E1              10        0.128205   
38548  51180               1             D3              26        0.487179   
53431  71135               1             D2              29        0.076923   
18262  24353               1             D1              26        0.230769   
11147  14794               1             D3              26        0.282051   
58236  77622               1             D2              26        0.384615   
37339  49586               1             D3              10        0.128205   
25533  34002               1             D2              26        0.487179   
11295  14987               1             D4              26        0.487179   
11842  15714               1             D3              26        0.487179   
12528  16653               1             A8              10        0.076923   
34853  46298               1             B2         

### Оценка модели

In [8]:
print ("kNN, 10:",
      cohen_kappa_score(data_test["target_10"], data_test["Response"],
                       weights="quadratic"))
print ("kNN, 100:",
      cohen_kappa_score(data_test["target_100"], data_test["Response"],
                       weights="quadratic"))
print ("kNN, 1000:",
      cohen_kappa_score(data_test["target_1000"], data_test["Response"],
                       weights="quadratic"))
print ("kNN, 10000:",
      cohen_kappa_score(data_test["target_10000"], data_test["Response"],
                       weights="quadratic"))
print ("kNN, max:",
      cohen_kappa_score(data_test["target_max"], data_test["Response"],
                       weights="quadratic"))

kNN, 10: 0.2957062865969361
kNN, 100: 0.30326951710809413
kNN, 1000: 0.28175078115596264
kNN, 10000: 0.15030991250481207
kNN, max: 0.28390472176878834


### Матрица неточностей

In [9]:
print (confusion_matrix(data_test["target_10"],
                       data_test["Response"]))
print (confusion_matrix(data_test["target_10000"],
                       data_test["Response"]))

[[ 209  162   19    9   84  224  153  148]
 [ 153  244   20    8  142  166   97  112]
 [   1    1    0    0    0    3    0    1]
 [   0    2    0    1    1    4    2   12]
 [  96  163   45    0  345  153   38   29]
 [ 243  274   54   49  226  690  382  411]
 [ 112  116   16   38   70  276  270  269]
 [ 371  344   58  207  208  768  673 2905]]
[[   0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0]
 [   1    1    0    0    0    0    0    0]
 [ 506  699  103   29  644  926  439  154]
 [   0    0    0    0    0    0    0    0]
 [ 678  606  109  283  432 1358 1176 3733]]
